In [ ]:
# Second version. This version is more efficient than the previous one 
# because the amount of joined data is reduced.
from pyspark.streaming import StreamingContext

In [ ]:
historicalInputFile = "data/Ex65/data/historicalData.txt"

In [ ]:
# Read the historical data and compute the maximum and minimum price for each stock
# Non-streaming RDD
historicalDataRDD = sc.textFile(historicalInputFile)

In [ ]:
# Return one pair (stockId, (price, price) )  for each input record
def extractStockIdPricePrice(line):
    fields = line.split(",")
    
    stockId = fields[1]
    price = fields[2]
    
    return (stockId, (float(price), float(price)) )




stockIdPriceHistoricalRDD = historicalDataRDD.map(extractStockIdPricePrice)

In [ ]:
# Compute max and min for each stockId based on the historical data
stockIdPriceHistoricalMaxMinRDD = stockIdPriceHistoricalRDD\
.reduceByKey(lambda v1, v2: ( max(v1[0],v2[0]), min(v1[1],v2[1]) ) ).cache()

In [ ]:
# Create a Spark Streaming Context object
#ssc = StreamingContext(sc, 60)
ssc = StreamingContext(sc, 5)

In [ ]:
# Create a (Receiver) DStream that will connect to localhost:9999
pricesDStream = ssc.socketTextStream("localhost", 9999)

In [ ]:
# Compute max and min for each stockId of each input window
# - windowDuration = 60 seconds
# - slideDuration = 30 seconds
stockIdPriceDStream = pricesDStream.map(extractStockIdPricePrice)\
.reduceByKeyAndWindow(lambda v1, v2: ( max(v1[0],v2[0]), min(v1[1],v2[1]) ),None\
                      ,10,5)
#                      ,60, 30)

In [ ]:
# Join stockIdPriceDStream with stockIdPriceHistoricalMaxMinRDD
# Join the RDD associated with the content of the current batch and 
# the non-streaming RDD stockIdPriceHistoricalMaxMinRDD
stockIdPriceMaxMinDStream = stockIdPriceDStream\
.transform(lambda batchRDD: batchRDD.join(stockIdPriceHistoricalMaxMinRDD))

In [ ]:
# Select only stocks with stream max price > maximum historical price 
# or stream min price < minimum historical price
def anomalyValue(pair):
    stockBatchMaxPrice = pair[1][0][0]
    stockBatchMinPrice = pair[1][0][1]
    
    stockHistoricalMaxPrice = pair[1][1][0]
    stockHistoricalMinPrice = pair[1][1][1]
    
    if stockBatchMaxPrice>stockHistoricalMaxPrice or stockBatchMinPrice<stockHistoricalMinPrice:
        return True
    else:
        return False

    

selectedStockPricesDStream = stockIdPriceMaxMinDStream\
.filter(anomalyValue)

In [ ]:
# Retrieve only the stockIDs of the selected stocks
# keys is not available for DStreams.
# transform must be used or map
selectStockIdsDStream = selectedStockPricesDStream\
.transform(lambda batchRDD: batchRDD.keys())

In [ ]:
selectStockIdsDStream.pprint()

In [ ]:
#Start the computation
ssc.start()

In [ ]:
# Run this application for 90 seconds
ssc.awaitTerminationOrTimeout(90)
ssc.stop(stopSparkContext=False)

In [ ]:
ssc.stop(stopSparkContext=False)